In [788]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
## basic functionality
import pandas as pd
import numpy as np
import re
import os
import statsmodels.api as sm
#import geopandas as gpd
## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import plotly.express as px
import datetime
import time
import janitor

import xgboost as xgb
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
from sklearn.linear_model import RidgeCV, LassoCV

from verstack.stratified_continuous_split import scsplit

import matplotlib.pyplot as plt
from stargazer.stargazer import Stargazer

In [511]:
df = pd.read_csv("../data/analysis_df.csv")

## Dataset Attributes

In [512]:
#unique values
df.id_cen.nunique()

# Dimensions
df.shape

6141

(325225, 124)

In [513]:
df = df[df['Total Population of Village']>0]

#we have per capita/percentage version of these variables so dropping these. 
df = df.drop(columns = ['Unnamed: 0','Total Geographical Area (in Hectares)',
 'Forest Area (in Hectares)',
 'Total   Households ',
 'Total Male Population of Village',
 'Total Female Population of Village',
 'Total Scheduled Castes Population of Village',
 'Total Scheduled Tribes Population of Village'])

## Variable Creation

### Per capita variables

In [514]:
df['spend_pcap'] = df.tot_spend/df['Total Population of Village']

### Per capita variables

In [515]:
#columns that have not been aggregated 
pcap_cols = [column for column in df.columns if '(Numbers)' in column]

In [516]:
#create per capita variables for these stand alone columns (for eg per capita mobile health clinics)
g = globals()
for i in pcap_cols:
    helper = re.sub(r'(.*)( \(Numbers\))',r'\1',i)
    
    df[helper+'_pcap'] = df[i]/df['Total Population of Village']

In [517]:
df_pcap = df.copy()

#dropping original version of unaggregated columns, since we now have their pcap version.
df_pcap.drop(pcap_cols,axis = 1,inplace = True)

#dropping text columns containing village names, block names etc and also dropping tot_spend, since we have spendpcap
#also dropping own spend since there are too many zeroes in the column for it to be meaningful.
df_pcap.drop(['sc_fund',
              'st_fund',
              'dist_cen',
              'blk_cen',
              'gp_cen',
              'id_cen',
              'dist_exp',
              'blk_exp',
              'gp_exp',
              'own_spend',
              'tot_spend',
              'Score'],axis = 1, inplace = True)

### Ranks 

We use ranks for the ambiguous variables.

### Demographic majority indicator

We create a categorical variable 'Majority Community', indicating the community which is in the majority in the GP. Its values are:

- 'Scheduled Castes', If SC population > 50%

- 'Scheduled Tribes', If ST population > 50%

- 'Non-Reserved', If (SC + ST) population < 50%, 

In [519]:
df_pcap['gen_perc'] = 1 - df_pcap['sc_perc']-df_pcap['st_perc']

#Fractionalization in villages
df_pcap['Fract'] = 1-df_pcap['sc_perc']**2-df_pcap['st_perc']**2-df_pcap['gen_perc']**2

### Training set 

#### one of our input features will be spend pcap for year x, and our output feature is mean(spendcap for year x+1 to x+4). We dont use outlier data for 2020 to 2021 due to covid, when all spending drops. 

#### We set aside 2022-2023 spend pcap data. We use this data on the machine trained on the above data to predict expenditures for 2024-2028

In [521]:
df_pcap_yr = df_pcap[(df_pcap.Plan_Year == '2015-2016') |(df_pcap.Plan_Year == '2016-2017') | (df_pcap.Plan_Year == '2017-2018') | (df_pcap.Plan_Year == '2018-2019') | (df_pcap.Plan_Year == '2019-2020') |  (df_pcap.Plan_Year == '2021-2022')]

In [904]:
#first we subset for our 5 year data 
df_pcap_5yr = df_pcap_yr[(df_pcap_yr.Plan_Year == '2016-2017') | (df_pcap_yr.Plan_Year == '2017-2018') | (df_pcap_yr.Plan_Year == '2018-2019') |  (df_pcap.Plan_Year == '2019-2020') |  (df_pcap_yr.Plan_Year == '2021-2022')]

#summing across all categories for a given year 
df_pcap_5yr = df_pcap_5yr[['id_exp','Plan_Year','spend_pcap']].groupby(['id_exp','Plan_Year']).sum().reset_index()

#taking the 5 year average for the summed spend_pcap
df_pcap_5yr = df_pcap_5yr.groupby(['id_exp']).mean().reset_index()

#There is some difference in what villages are in a particular year. So for 2015-2016 we 
#subset for villages that exist between 2016-2021, i.e. exist in df_pcap_5yr
df_pcap_15 = df_pcap_yr[(df_pcap_yr.id_exp.isin(df_pcap_5yr.id_exp.unique().tolist()))]

#villages are selected, subset for year 
df_pcap_15 = df_pcap_15[df_pcap_15.Plan_Year == '2015-2016']

#sum across all categories for that year 
df_pcap_15 = df_pcap_15[['id_exp','Plan_Year','spend_pcap']].groupby('id_exp').sum().reset_index()

In [940]:
df_pcap_15.rename(columns = {'spend_pcap':'spend_pcap_15'}, inplace = True)
df_pcap_5yr.rename(columns = {'spend_pcap':'spend_pcap_avg'}, inplace = True)

#Note: Since the rest of the input features are going to be the same (demographic data from 2011), 
#we perform the groupby aggregations only for spend_pcap

df_pcap_15.head()
df_pcap_5yr.head()

,id_exp,spend_pcap_15
0,anugul_anugul_angarbandha,400.535236
1,anugul_anugul_antula,315.509915
2,anugul_anugul_badakantakul,304.862646
3,anugul_anugul_badakera,265.601322
4,anugul_anugul_balanga,679.909194


,id_exp,spend_pcap_avg
0,anugul_anugul_angarbandha,714.510660
1,anugul_anugul_antula,576.988987
2,anugul_anugul_badakantakul,505.191380
3,anugul_anugul_badakera,452.006365
4,anugul_anugul_balanga,1148.932047


In [906]:
#creating final dataframe to be used as input in machine
spends = df_pcap_15.merge(df_pcap_5yr, on = 'id_exp', how = 'left')
df_cen = df_pcap_yr[df_pcap_yr.Plan_Year == '2015-2016'].groupby('id_exp').mean().reset_index()
df_cen.drop(columns = ['spend_pcap','n_act','sc_perc','st_perc','gen_perc'],inplace = True)

df_pcap_final = df_cen.merge(spends, on = 'id_exp', how = 'right')

In [941]:
df_pcap_final.head()

,id_exp,Total Population of Village,roads_sum,fin_sum,power_hrs_sum,forest_perc,fem_to_male,Community Health Centre_pcap,Community Health Centre Doctors Total Strength_pcap,Community Health Centre Doctors In Position_pcap,...,Private Vocational Training School/ITI_pcap,Government Non Formal Training Centre_pcap,Private Non Formal Training Centre_pcap,Government School For Disabled_pcap,Private School For Disabled_pcap,Government Others_pcap,Private Others_pcap,Fract,spend_pcap_15,spend_pcap_avg
0,anugul_anugul_angarbandha,4484.0,9.0,3.0,76.0,0.006200,0.931525,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.242961,400.535236,714.510660
1,anugul_anugul_antula,5648.0,17.0,9.0,26.0,0.326379,0.985589,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628919,315.509915,576.988987
2,anugul_anugul_badakantakul,6334.0,29.0,5.0,0.0,0.259894,0.957664,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538105,304.862646,505.191380
3,anugul_anugul_badakera,7259.0,32.0,8.0,20.0,0.084902,0.944296,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.468553,265.601322,452.006365
4,anugul_anugul_balanga,2643.0,12.0,5.0,0.0,0.174216,1.026054,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.548151,679.909194,1148.932047


### Methodology and Architecture

### Gradient Boosting (Xgboost)

In [397]:
#The values we experiment with for a regularized gradient boosting using xgboost 
reg_alpha = [int(x) for x in np.linspace(start = 1, stop = 20, num = 1)]
learning_rate = [0.1,0.2,0.3,0.4,0.5]
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 3000, num = 100)]
gamma = [int(x) for x in np.linspace(start = 0, stop = 15, num = 1)]
max_depth = [int(x) for x in np.linspace(start = 5, stop = 10, num = 1)]

random_grid = {'reg_alpha': reg_alpha,
               'learning_rate': learning_rate,
               'n_estimators': n_estimators,
               'gamma': gamma,
               'max_depth': max_depth,}

#Best values we found for xgboost. The results we still subpar
best = {'gamma': [0],
 'learning_rate': [0.2],
 'max_depth': [5],
 'n_estimators': [158],
 'reg_alpha': [1]}

In [907]:
X = np.array(df_pcap_final.iloc[:,1:101])
y = np.array(df_pcap_final.iloc[:,101])
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.3,random_state = 5)


xgb_base = xgb.XGBRegressor(objective="reg:squarederror", 
                            random_state=42,
                            learning_rate = 0.3,
                            n_estimators = 500,
                            gamma = 10,
                            reg_alpha = 8)

# xgb_model = GridSearchCV(
#                         estimator = xgb_base, 
#                         param_grid = random_grid, 
#                         cv = 5, 
#                         n_jobs = -1, 
#                         verbose = 2)

xgb_base.fit(X_train, y_train) #training xgboost
y_pred = xgb_base.predict(X_test) #testing on trained model

print('MSE: %.3f' % mean_squared_error(y_test,y_pred))
print('Training accuracy: %.3f' % xgb_base.score(X_train,y_train))
print('Testing accuracy: %.3f' % xgb_base.score(X_test, y_test))

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=10, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.3, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=42, ...)

MSE: 1236963734.887
Training accuracy: 1.000
Testing accuracy: 0.139


### Lasso Regression

In [942]:
# lasso_model = Lasso().fit(X_train,y_train)
lasso_tuned = Lasso().set_params(alpha = 1).fit(X_train,y_train)

y_pred = lasso_tuned.predict(X_test)
print('Training accuracy: %.3f' % lasso_tuned.score(X_train,y_train))
print('Testing accuracy: %.3f' % lasso_tuned.score(X_test, y_test))
print('Testing R2: %.3f' %r2_score(y_test, y_pred))

Training accuracy: 0.904
Testing accuracy: 0.853
Testing R2: 0.853


In [943]:
r2_plot = pd.DataFrame(list(zip(y_pred,y_test))).rename(columns = {0:'pred',1:'test'})
r2_plot['res'] = r2_plot['pred']- r2_plot['test']

## Need to plot r2 against predictor to check for residual normality 

In [903]:
## coefficient importance 
## Note: none of the variables have been standardized/log so interpretat as is 

feature_list = list(df_pcap_final.iloc[:,1:101].columns)
df = pd.DataFrame(list(zip(feature_list, lasso_tuned.coef_ )),
               columns =['Name', 'val'])
df[df['val']!=0]

,Name,val
0,Total Population of Village,-0.073378
1,roads_sum,-0.898911
2,fin_sum,-2.748364
3,power_hrs_sum,-0.157292
4,forest_perc,-233.062529
98,Fract,-494.328784
99,spend_pcap_15,0.839270


### Visualizing results


### Predicting for 2024-2028 5 yr average 

In [936]:
#creating prediction data frame 
df_pred = df_pcap[['id_exp','spend_pcap']][df_pcap.Plan_Year == '2022-2023'].groupby('id_exp').sum().reset_index()
df_cen_23 = df_pcap[df_pcap.Plan_Year == '2022-2023'].groupby('id_exp').mean().reset_index()
df_cen_23.drop(columns = ['spend_pcap','n_act','sc_perc','st_perc','gen_perc'],inplace = True)
df_pred_final = df_cen_23.merge(df_pred, on = 'id_exp', how = 'right')
df_pred_final

,id_exp,Total Population of Village,roads_sum,fin_sum,power_hrs_sum,forest_perc,fem_to_male,Community Health Centre_pcap,Community Health Centre Doctors Total Strength_pcap,Community Health Centre Doctors In Position_pcap,...,Govt Vocational Training School/ITI_pcap,Private Vocational Training School/ITI_pcap,Government Non Formal Training Centre_pcap,Private Non Formal Training Centre_pcap,Government School For Disabled_pcap,Private School For Disabled_pcap,Government Others_pcap,Private Others_pcap,Fract,spend_pcap
0,anugul_anugul_angarbandha,4484.0,9.0,3.0,76.0,0.006200,0.931525,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.242961,738.359054
1,anugul_anugul_antula,5648.0,17.0,9.0,26.0,0.326379,0.985589,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628919,586.198654
2,anugul_anugul_badakantakul,6334.0,29.0,5.0,0.0,0.259894,0.957664,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.538105,522.710767
3,anugul_anugul_badakera,7259.0,32.0,8.0,20.0,0.084902,0.944296,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.468553,456.102769
4,anugul_anugul_balanga,2643.0,12.0,5.0,0.0,0.174216,1.026054,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.548151,993.106319
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6562,sundargarh_tangarpali_remenda,2585.0,14.0,4.0,0.0,0.079939,1.028235,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.650789,900.318762
6563,sundargarh_tangarpali_sanpatrapali,7013.0,30.0,6.0,0.0,0.060043,1.016097,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.401679,420.078711
6564,sundargarh_tangarpali_tangarpali,4251.0,16.0,4.0,41.0,0.023095,1.018993,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.605245,547.476829
6565,sundargarh_tangarpali_tasladihi,4512.0,19.0,5.0,0.0,0.022821,1.031053,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.569304,515.807624


In [938]:
#predicting on trained model
X_pred = df_pred_final.iloc[:,1:101]
y_pred = lasso_tuned.predict(X_pred)
y_pred

array([1327.85966454,  787.27655907,  749.49947161, ...,  958.22647366,
        934.90468033,  551.7111082 ])